In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [43]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Assuming df is your DataFrame and 'Feature17' has missing values
imputer = IterativeImputer(max_iter=100, tol=1e-3)

df_train = pd.read_csv('../data/iith_foml_2023_train.csv')
# Fit the imputer model on the dataset to learn the data patterns
# df_train.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
# imputer.fit(df_train)

# # Transform the dataset to replace missing values
# df_imputed = imputer.transform(df_train)

# # Convert back to a DataFrame
# df_imputed = pd.DataFrame(df_imputed, columns=df_train.columns)

for i in df_train['Target Variable (Discrete)'].unique():
    df_train['Feature 16'].loc[df_train['Target Variable (Discrete)'] == i].fillna(df_train['Feature 16'].loc[df_train['Target Variable (Discrete)'] == i].mean(), inplace=True)
    df_train['Feature 17'].loc[df_train['Target Variable (Discrete)'] == i].fillna(df_train['Feature 17'].loc[df_train['Target Variable (Discrete)'] == i].mean(), inplace=True)
df_imputed = df_train

In [44]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

X_train = df_imputed.iloc[:, :-1]
y_train = df_imputed.iloc[:, -1]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
class_wts = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_wts_dict = dict(zip(np.arange(len(class_wts)), class_wts))
sample_wts = compute_sample_weight(class_weight=class_wts_dict, y=y_train)

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score

def report_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro', zero_division=0.0)

# Initialize model
model = XGBClassifier(random_state=42, n_estimators=200)

# Train the model on the training data
cv_f1_score = cross_val_score(model, X_train, y_train, scoring=make_scorer(report_f1), error_score='raise')
print(cv_f1_score)


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15], got [ 0  1  2  3  4  5  6  7  8  9 10 13 14 15 16 17]